# Deterministic POS Word Level Language Model with Penn Treebank
Sequence Tagger: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html<br>
Penn Treebank: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.9.8216&rep=rep1&type=pdf

https://gist.github.com/williamFalcon/f27c7b90e34b4ba88ced042d9ef33edd <br>
https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

In [1]:
import numpy as np
import nltk
from nltk.corpus import treebank
import os
import codecs

In [2]:
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /home/tyler/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/tyler/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(1)

In [4]:
from IPython.display import clear_output

# An LSTM for Part-of-Speech Tagging

## Create DataLoader
Something something data loader...

In [128]:
class PennTreeBankDataset(Dataset):
    """Penn Tree Bank dataset"""
    
    def __init__(self):
        # download data, read, process
        np_array = np.zeros((1000,10))
        
        # load Penn tree bank sentences
        self.sentences = treebank.tagged_sents(tagset='universal')
        self.sentences = [self.format_sequence(sentence) for sentence in self.sentences]
        self.len = len(self.sentences)
        
        self.pad_token = '<PAD>'
        self.pad_token_id = 0
        self.pad_len = max([len(sentence[0]) for sentence in self.sentences])
        
        # Vocab of input data (this will likely be a subset of any word embedding array)
        self.build_vocab()
        self.create_word_to_idx_dict()
        self.create_tag_to_idx_dict()
        self.X, self.y, self.X_lens = self.encode_and_pad_data()
        
    def format_sequence(self,seq):
        """
        Formats penn treebank POS format into tuple ([tokens], [POS])
        """
        tokens = [x[0] for x in seq]
        tags = [x[1] for x in seq]
        return (tokens, tags)
    
    def build_vocab(self):
        """Builds vocab dictionary based on input data"""
        self.vocab = dict()
        for sentence in self.sentences:
            for word in sentence[0]:
                if word not in self.vocab:
                    self.vocab[word] = len(self.vocab) + 1    # counts from 1+ as 0 is reserved for PAD token
        
        # Add padding token to data vocab
        self.vocab[self.pad_token] = 0
    
    def create_word_to_idx_dict(self):
        """Builds word to index dictionary"""
        self.word_to_idx = {}
        for sentence, tags in self.sentences:
            for word in sentence:
                if word not in self.word_to_idx:
                    self.word_to_idx[word] = len(self.word_to_idx) + 1    # counts from 1+ as 0 is reserved for PAD token
        
        # add padding to word dict
        self.word_to_idx[self.pad_token] = 0
        
    def create_tag_to_idx_dict(self):
        """Builds tag to index and index to tag dictionary"""
        self.tag_to_idx = {}
        for _, tags in self.sentences:
            for tag in tags:
                if tag not in self.tag_to_idx:
                    self.tag_to_idx[tag] = len(self.tag_to_idx) + 1    # counts from 1+ as 0 is reserved for PAD token

        # add padding to tag dict
        self.tag_to_idx[self.pad_token] = 0
        self.idx_to_tag = {v:k for k, v in self.tag_to_idx.items()}
        
    def encode_and_pad_data(self):
        """Encodes data (seq and tags) into ids from id dictionary and pads"""
        batch_size = len(self.sentences)

        padded_seq_batch = np.full((batch_size, self.pad_len), self.pad_token_id)
        padded_tags_batch = np.full((batch_size, self.pad_len), self.pad_token_id)
        seq_lengths = [len(sentence[0]) for sentence in self.sentences]
        
        # encode and pad
        for i, (seq, tags) in enumerate(self.sentences):
            # encode
            seq_idxs = [self.word_to_idx[token] for token in seq]
            tags_idxs = [self.tag_to_idx[tag] for tag in tags]        
            # pad
            padded_seq_batch[i,0:len(seq_idxs)] = seq_idxs
            padded_tags_batch[i, 0:len(tags_idxs)] = tags_idxs
            
        return torch.tensor(padded_seq_batch, dtype=torch.long), torch.tensor(padded_tags_batch, dtype=torch.long), torch.tensor(seq_lengths, dtype=torch.int)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index], self.X_lens[index]
    
    def __len__(self):
        return self.len

In [129]:
dataset = PennTreeBankDataset()

In [130]:
train_loader = DataLoader(dataset=dataset,
                         batch_size=32,
                         shuffle=True,
                         num_workers=2)

### Create LSTM model

### Initialise Word Embeddings

In [132]:
path_to_embeddings = './data/embeddings/glove.6B.300d.txt'
path_to_trimmed_embeddings = './data/embeddings/trimmed_emb.npz'

### Initialse Model

In [337]:
class LSTMTagger(nn.Module):
    def __init__(self,
                 embedding_dim,
                 hidden_dim,
                 vocab,
                 tags,
                 batch_size=32,
                 pretrained_embeddings=None):
        
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = 1
        self.embedding_dim = embedding_dim
        self.vocab = vocab
        self.vocab_size = len(self.vocab)
        self.tags = tags
        self.tagset_size = len(self.tags)
        self.tagset_size = self.tagset_size - 1 # minus <TAG>
        self.padding_idx = self.vocab['<PAD>']
        
        self.word_embeddings = nn.Embedding(num_embeddings=self.vocab_size,
                                            embedding_dim=self.embedding_dim,
                                            padding_idx=self.padding_idx)
        
        if pretrained_embeddings is not None:
            self.word_embeddings.weight.data.copy_(torch.from_numpy(pretrained_embeddings))
            self.word_embeddings.weight.requires_grad = False
        
        self.hidden = self.init_hidden()

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        # If batch_first is true the input/output tensors are provided as (batch, seq, feature)
        self.lstm = nn.LSTM(input_size=self.embedding_dim,
                            hidden_size=self.hidden_dim,
                            num_layers=self.num_layers,
                            batch_first=True)

        # The linear layer that maps from hidden state space to tag space
        self.out = nn.Linear(in_features=self.hidden_dim,
                             out_features=self.tagset_size)

    def init_hidden(self):
        """
        Initialises weights for hidden layers of LSTM
        Weights are in the form of (num_layers, batch_size, embedding_dim)"""
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim), torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
        
    def forward(self, X, seq_lengths):
        """"""
        print('X input shape', X.shape)
        print('seq_lengths shape:', max(seq_lengths))
        
        self.hidden = self.init_hidden()
        
        batch_size, seq_len = X.size()
        
        
        # Embed the input
        # (batch_size, seq_len) -> (batch_size, seq_len, embedding_dim)
        embedded_seq_tensor = self.word_embeddings(X)
        print('Embedding seq tensor shape:', embedded_seq_tensor.shape)
        
        
        # --- Run through model ---
        # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(embedded_seq_tensor, seq_lengths, batch_first=True, enforce_sorted=False)   # TODO: enfore sorting...
        print('Packed input shape:', packed_input.data.shape)
        
        
        # run through LSTM
        packed_output, self.hidden = self.lstm(packed_input, self.hidden)
        
        
        # undo the packing operation
        output, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        print('X size after passing through LSTM:', output.shape)
        
#         print(output[:5])
        
        # --- Project to tag space ---
        # (batch_size, seq_len, embedding_dim) -> (batch_size * seq_len, embedding_dim)
        output = output.contiguous()
        output = output.view(-1, output.shape[2])
        print('X after reshaping:', output.shape)
        
        
        # Run through linear layer
        output = self.out(output)
        
        
        # Perform softmax
        # (batch_size * seq_len, embedding_dim) -> (batch_size, seq_len, tagset_size)
        output = F.log_softmax(output, dim=1)
        print('X shape after softmax:', output.shape)
        
        
        # Reshape for sanity
        output = output.view(batch_size, max(seq_lengths), self.tagset_size)
        print('X after sanity reshape:', output.shape)
        
        
        Y_hat = output
        return Y_hat
    
        
    def loss(self, Y_hat, Y, seq_lengths):
        """"""
        print('\n--- LOSS FUNCTION ---\n')
        
        ymask = ~torch.all(Y == self.tags['<PAD>'], dim=1)
        
        Y = Y[ymask]
        Y_hat = Y_hat[ymask]
        
        
        # Unsure if this is meant to work, but truncate all Y by max(seq_length) so
        # view size matches that of Y_hat due to pad packing
        print('Max seq length:',max(seq_lengths))
        # truncate Y by max seq length (similar to forward process)
        Y = Y[:,:max(seq_lengths)]
        
        # Flatten all the target labels
        print('Y (pre) shape:', Y.shape)
#         Y = Y.view(-1)   # .to(device)
        Y = Y.reshape(-1)
        print('Y (post) shape:', Y.shape)
        
        
        # Flatten all the predicted labels
        Y_hat = Y_hat.view(-1, self.tagset_size)
        print('Y_hat shape:', Y_hat.shape)
        
        print(Y_hat)
        
        tag_pad_token = self.tags['<PAD>']    # should be 0
        mask = (Y > tag_pad_token).float()
            
        # Count how many tokens there are
        num_tokens = int(torch.sum(mask).item())
        print('Number of tokens:',num_tokens)
        
#         print(Y_hat.shape[0])
        
        # Pick the values for the label and zero out the rest with the mask
        Y_hat = Y_hat[range(Y_hat.shape[0]), Y] * mask
        
        ce_loss = -torch.sum(Y_hat) / num_tokens
        
        return ce_loss

In [338]:
# Load trimmed embeddings from disk
pretrained_embeddings = np.load(path_to_trimmed_embeddings)

In [339]:
EMBEDDING_DIM = 300   # Using Glove 300 dim
HIDDEN_DIM = 16

In [340]:
lstm_net = LSTMTagger(embedding_dim=EMBEDDING_DIM,
                      hidden_dim=HIDDEN_DIM,
                      vocab=dataset.vocab,
                      tags=dataset.tag_to_idx,
                      pretrained_embeddings=pretrained_embeddings['embeddings'])
loss_function = nn.NLLLoss()
optimizer = optim.SGD(lstm_net.parameters(), lr=0.1)

In [341]:
# print(lstm_net)

### Train standard NN model

In [342]:
for epoch in range(1):
    for batch_idx, (data, targets, data_lens) in enumerate(train_loader, 0):
    
        lstm_net.zero_grad()
        
        # forward pass...
        tag_scores = lstm_net(data, data_lens)

        # compute loss, gradients and update parameters by calling optimzier.step()
        loss = lstm_net.loss(tag_scores, targets, data_lens)
        
        break

X input shape torch.Size([32, 271])
seq_lengths shape: tensor(53, dtype=torch.int32)
Embedding seq tensor shape: torch.Size([32, 271, 300])
Packed input shape: torch.Size([891, 300])
X size after passing through LSTM: torch.Size([32, 53, 16])
X after reshaping: torch.Size([1696, 16])
X shape after softmax: torch.Size([1696, 12])
X after sanity reshape: torch.Size([32, 53, 12])

--- LOSS FUNCTION ---

Max seq length: tensor(53, dtype=torch.int32)
Y (pre) shape: torch.Size([32, 53])
Y (post) shape: torch.Size([1696])
Y_hat shape: torch.Size([1696, 12])
tensor([[-2.4141, -2.2352, -2.3570,  ..., -2.5385, -2.6173, -2.5240],
        [-2.4653, -2.2464, -2.3432,  ..., -2.5027, -2.7113, -2.4663],
        [-2.3957, -2.2573, -2.3320,  ..., -2.5000, -2.6744, -2.4351],
        ...,
        [-2.4403, -2.2441, -2.3324,  ..., -2.5302, -2.6240, -2.4988],
        [-2.4403, -2.2441, -2.3324,  ..., -2.5302, -2.6240, -2.4988],
        [-2.4403, -2.2441, -2.3324,  ..., -2.5302, -2.6240, -2.4988]],
       gr

IndexError: index 12 is out of bounds for dimension 1 with size 12

# Old code

In [ ]:
for epoch in range(2):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        lstm_net.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = lstm_net(sentence_in, len(sentence_in))

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
    
    if epoch % 1 == 0:
        print(f'Epoch: {epoch} - Loss: {loss}')

In [ ]:
# helper function for deterministic nn inference
def tag_score_to_tag_name(tag_score, ix_to_tag):
    """Converts tag score to tag names"""
    return ix_to_tag.get(torch.argmax(tag_score).item())

In [ ]:
# Single test example
test_data_sm = test_data[:1]

In [ ]:
# Inference
with torch.no_grad():
    inputs = prepare_sequence(test_data_sm[0][0], word_to_ix)
    tag_scores = lstm_net(inputs)
    
#     print(f'Tag Scores:\n{tag_scores}\n')
    print(f'{"Token":<20} {"Pred":<10} {"Actual":<10}')
    print(f'{"-----":<20} {"----":<10} {"------":<10}')
    for i, token in enumerate(training_data[0][0]):
        print(f'{token:<20} {tag_score_to_tag_name(tag_scores[i], ix_to_tag):<10} {test_data_sm[0][1][i]:<10}')